In [1]:
#!/usr/bin/env python3
from twitter import *
import json

consumer_key = "vAm5pdHyfy1xbTdwGtTP1A"
consumer_secret = "7Kp9IIvYbseJzpknZPOXjcXQHfXsSG4hLUFZ9iz1Ws"
access_key = "1448875278-tFuv7p6heGXhCk5xfDQ0Btqh5sFkVkRJbodOUyI"
access_secret = "SHm0LQtrwzHgpqkudPwXIJWpzZdvUAGTwBq5HSlxOzYFF"

tweetidfilename='sanders.txt'

auth = OAuth(access_key, access_secret, consumer_key, consumer_secret)
twitter = Twitter(auth = auth)

tweetids=[e.strip() for e in open(tweetidfilename).readlines()]

# LIMIT TO 900 TWEETS B/C OF API RESTRICTIONS AND CREATE CHUNKS OF 100
def chunker(seq, size):
    return [seq[pos:pos + size] for pos in range(0, len(seq), size)]
batches = chunker(tweetids[:900],100)

# API expects comma-separated strings, no lists:
batches = [','.join(l) for l in batches]

posts = []

for batch in batches:
    posts += twitter.statuses.lookup(_id=batch)


headers=["text","retweeted","retweet_count","hashtags","urls","created_at","description","screen_name","name","friends_count","followers_count","statuses_count"]
with open('sanders.tsv',mode='w') as fo:
        fo.write("\t".join(headers)+"\n")
        for post in posts:
            output=[]
            output.append(post["text"].replace("\t"," ").replace("\n"," ").replace("\r"," "))
            output.append(str(post["retweeted"]))
            output.append(str(post["retweet_count"]))            
            hashtags = [tag["text"] for tag in post["entities"]["hashtags"]]        
            output.append(str(hashtags))
            urls = [u["expanded_url"] for u in post["entities"]["urls"]]    
            output.append(str(urls))    
            output.append(post["created_at"])
            output.append(post["user"]["description"].replace("\t"," ").replace("\n"," ").replace("\r"," "))
            output.append(post["user"]["screen_name"])
            output.append(post["user"]["name"].replace("\t"," ").replace("\n"," ").replace("\r"," "))
            output.append(str(post["user"]["friends_count"]))
            output.append(str(post["user"]["followers_count"]))
            output.append(str(post["user"]["statuses_count"]))            
            fo.write("\t".join(output)+"\n")


In [4]:
import csv
tweetlist = []
processedlist = []

with open('/Users/shujun/Code/Python/Sentiment/Sanders.tsv')as fi:
    reader = csv.reader(fi, delimiter='\t')
    next(reader)##skip the header row with the next()
    for row in reader:
        tweet = row[0]
        tweet_processed=tweet.lower().replace("!"," ").replace("."," ").replace("?"," ").replace("'"," ").replace('"'," ").replace("#"," ").replace(':'," ")
        tweetlist.append(tweet)
        processedlist.append(tweet_processed)

In [7]:
from pprint import pprint
pprint(tweetlist)

['Why Bernie Sanders Got Twice as Much Applause as Hillary Clinton When He '
 'Spoke to La Raza @alternet http://t.co/ezknEeRIuU',
 'Notable signees to letter, led by Robert Menendez, include not only '
 'Elizabeth Warren and Bernie Sanders, but Marco Rubio and Rob Portman.',
 "Bernie Sanders gets his say on Clinton's day in Capitol: WASHINGTON (AP) — "
 'Former Secretary of State Hillary R... http://t.co/LRd5Y12eyB',
 'Bernie Sanders gaining on Hillary Clinton, with Biden looming: poll '
 'http://t.co/M8Wd9QSBC3 via @washtimes',
 "RT @ungerbn103: @SenSanders @stavernise  Tragically, that concern doesn't "
 'seem to be on the radar of the GOP.',
 'RT @tfarley1969: @CMZR @TheBoNose @SenatorTester @LIOptOut @UFT @SenSanders '
 'and LISTEN to the parents!',
 'RT @FuelTheBern: Hillary Clinton is all words. Bernie Sanders has backed his '
 'words with real legislation. #FeelTheBern! #Bernie2016!  http:…',
 'Watch Ur Beck RT @tacotansel I want Bernie Sanders to be president so Donald '
 'Tru

In [19]:
positivelist = open("/Users/shujun/Code/Python/Sentiment/positive-words.txt").read().splitlines()

In [12]:
negativelist = open("/Users/shujun/Code/Python/Sentiment/negative-words.txt", encoding = "ISO-8859-1").read().splitlines()

In [29]:
negcountlist = []
poscountlist = []
sentilist = []

for tweet in processedlist:
    poscount = 0
    negcount = 0
#     print("Analyzing this one:",tweet)
    
    for word in tweet.split():
        if word in positivelist:
            poscount+=1
        elif word in negativelist:
            negcount+=1
#     print("It contains", poscount, "positive words and", negcount, "negative words.")
#     print("--------")
    
    poscountlist.append(poscount)
    negcountlist.append(negcount)
    sentilist.append((poscount-negcount)/len(tweet.split()))
print("Average sentiment:",sum(sentilist)/len(sentilist))

Average sentiment: 0.010680186833531322
